<a href="https://colab.research.google.com/github/HARJAP-SINGH-3105/Is_the_road_safe-/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
import gc

In [3]:
from tensorflow import keras
import numpy as np 
import cv2
from PIL import Image
from matplotlib import pyplot as plt


In [4]:
from keras.datasets import fashion_mnist

In [5]:
(X_train,Y_train),(X_test,Y_test) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
X_train = X_train[:40000].astype(np.float32)/255.0


In [7]:
X_test = X_test[:10000].astype(np.float32)/255.0

In [8]:
print(X_train[0].shape)

(28, 28)


In [9]:
from keras.models import Sequential
from keras.layers import Flatten,Dense,GlobalAveragePooling2D

In [13]:
#Built Model with Transfer Learning (ResNet50)
resnet_model =Sequential()
base_model = tf.keras.applications.ResNet50(
    include_top = False,
    input_shape = (32,32,3),
    weights='imagenet',
    input_tensor= None,
    pooling ='avg',
    classes =10
)

for layer in base_model.layers:
  layer.trainable = False

In [14]:

resnet_model.add(base_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(128,activation='relu'))
resnet_model.add(Dense(10,activation='softmax'))
# resnet_model.summary(_)

In [15]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 23,851,274
Trainable params: 263,562
Non-trainable params: 23,587,712
_________________________________________________________________


In [16]:
from tensorflow.keras.optimizers import Adam

In [17]:
resnet_model.compile(optimizer= Adam(learning_rate =0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [18]:
size = (32,32)

def resize_the_images(data):
  array  = np.zeros((data.shape[0],32,32))
  for i in range(0,data.shape[0]):
 
    array[i,:,:] =cv2.resize(data[i],size,interpolation =cv2.INTER_AREA)

 
  return array
    


In [19]:
train_data  = resize_the_images(X_train)
test_data = resize_the_images(X_test)
del X_train
del X_test
gc.collect()

19400

In [20]:
train_data =  train_data.reshape((train_data.shape[0],32,32,1))
test_data = test_data.reshape((test_data.shape[0],32,32,1))
# train_data[0,15:,10:,:]

In [21]:
train_arr= np.zeros((len(train_data),32,32,3))
for i, item in enumerate(train_data):
  train_arr[i,:,:,:]= cv2.cvtColor(item.astype(np.float32),cv2.COLOR_GRAY2BGR)

test_arr = np.zeros((len(test_data),32,32,3))
for i, item in enumerate(test_data):
  test_arr[i,:,:,:]= cv2.cvtColor(item.astype(np.float32),cv2.COLOR_GRAY2BGR)

In [22]:
train_arr.shape

(40000, 32, 32, 3)

In [23]:
# train_data = train_data.reshape((train_data.shape[0], 32, 32, 3))
# test_data = test_data.reshape((test_data.shape[0], 32,32, 3))

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator()
# test_datagen = ImageDataGenerator(rescale=1)

In [25]:
# train_datagen = ImageDataGenerator(rescale=1./255)
# # test_datagen = ImageDataGenerator(rescale=1./255)
# train_arr[0,20:,5:,:]

In [26]:
Y_train = Y_train[:40000]
Y_test =Y_test[:10000]

In [27]:
# Y_train.shape

In [28]:
Y_train =Y_train.reshape(Y_train.shape[0],1)
Y_test =Y_test.reshape(Y_test.shape[0],1)


In [29]:
epochs = 5
# Batch size
batch_size = 64
history =  resnet_model.fit(train_datagen.flow(train_arr,Y_train,
                                       batch_size=batch_size, 
                                       seed=27
                                       ),
                    epochs=epochs,
                    steps_per_epoch=train_arr.shape[0]//batch_size,
                    validation_data=(test_arr,Y_test),
                    verbose=2)

Epoch 1/5
625/625 - 171s - loss: 1.5683 - accuracy: 0.0821 - val_loss: 1.2156 - val_accuracy: 0.0710 - 171s/epoch - 273ms/step
Epoch 2/5
625/625 - 167s - loss: 1.0896 - accuracy: 0.0865 - val_loss: 1.0107 - val_accuracy: 0.0917 - 167s/epoch - 267ms/step
Epoch 3/5


KeyboardInterrupt: ignored

In [30]:
 history = resnet_model.fit(train_arr,Y_train,epochs=5,validation_data = (test_arr,Y_test),verbose = 2)

Epoch 1/5
1250/1250 - 187s - loss: 0.8783 - accuracy: 0.0978 - val_loss: 0.8536 - val_accuracy: 0.0874 - 187s/epoch - 149ms/step
Epoch 2/5


KeyboardInterrupt: ignored

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
# plt.show()
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()


In [ ]:
=